In [60]:
import pandas as pd
import numpy as np

In [61]:
df_raw = pd.read_csv("../resources/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
df_raw.columns=["unixtimestamp", "open", "high","low","close", "volume_btc","volume_usd","weighted_price"]

In [62]:
df = df_raw.dropna()
df = df.loc[(df["volume_btc"] > 0) & (df["high"] != df["low"])]
df.insert(loc=0, column='timestamp', value=pd.to_datetime(df['unixtimestamp'],unit = 's'))
since = np.datetime64("2018-01-01")
df = df.loc[df["timestamp"] > since]
df.reset_index(inplace=True, drop=True)

In [63]:
df.dtypes

timestamp         datetime64[ns]
unixtimestamp              int64
open                     float64
high                     float64
low                      float64
close                    float64
volume_btc               float64
volume_usd               float64
weighted_price           float64
dtype: object

In [64]:
df.head()

,timestamp,unixtimestamp,open,high,low,close,volume_btc,volume_usd,weighted_price
0,2018-01-01 00:01:00,1514764860,13841.01,13890.43,13841.01,13890.43,1.665850,23064.082359,13845.237237
1,2018-01-01 00:02:00,1514764920,13846.64,13890.35,13846.64,13847.51,0.269346,3739.690844,13884.349123
2,2018-01-01 00:03:00,1514764980,13847.51,13874.92,13823.19,13823.19,1.155274,15986.279494,13837.650132
3,2018-01-01 00:04:00,1514765040,13823.19,13826.75,13803.20,13803.20,0.646691,8930.465767,13809.482390
4,2018-01-01 00:05:00,1514765100,13826.10,13826.10,13734.45,13737.93,38.030409,524066.713280,13780.201774


In [81]:
class Backtester:
    def __init__(self, historical):
        self.historical = historical
    
    def test(self, usd_amount, btc_amount, agent):
        hist_iter = self.historical.iterrows()
        _, candle_prev = next(hist_iter)
        _, candle_curr = next(hist_iter)
        ctx = None
        while True:
            price = candle_curr["open"]
            ctx, signal = agent(ctx, usd_amount, btc_amount, candle_prev, price)
            if usd_amount < price * signal:
                raise Exception(f"Insufficient USD: {usd_amount} < {price * signal}")
            if btc_amount < -signal:
                raise Exception(f"Insufficient BTC: {btc_amount} < {-signal}. Agent failed")
            usd_amount -= price * signal # if -0.1 btc, that means usd amount increases
            btc_amount += signal
            date = candle_curr["timestamp"]
            h, m, s = date.hour, date.minute, date.second
            if h + m + s == 0:
                print(f"date={date}, price={price}, signal={signal}, usd_amount={usd_amount}, btc_amount={btc_amount}")
            candle_prev = candle_curr
            _, candle_curr = next(hist_iter)
            if candle_curr is None:
                return usd_amount + candle_prev["open"] * btc_amount

In [90]:
prev = df.iloc[10]["timestamp"]
print(prev)
curr = df.iloc[100]["timestamp"]
print(curr)
(curr - prev).total_seconds() > 3600

2018-01-01 00:11:00
2018-01-01 01:42:00


True

In [82]:
def agent(ctx, usd_amount, btc_amount, candle_prev, timestamp, price):
    if ctx is None:
        ctx = {"last_operation": candle_prev["timestamp"]}
        return 0
    if (ctx["last_operation"] - timestamp).total_seconds() < 3600:
        return 0
    ctx["last_operation"] = timestamp
    if uptrend:
        return (usd_amount / price) / 20
    if downtrend:
        return -btc_amount / 20
    return 0

In [83]:
bkt = Backtester(df)

In [84]:
bkt.test(1000, 0, agent)

date=2018-01-02 00:00:00, price=13394.2, signal=0.0019214507661802637, usd_amount=488.98962119506206, btc_amount=0.037288711342039434
date=2018-01-03 00:00:00, price=14670.96, signal=-0.0018046660456744377, usd_amount=552.0447482454352, btc_amount=0.034288654867814315
date=2018-01-04 00:00:00, price=15155.62, signal=-0.0017058576073440313, usd_amount=588.923628311066, btc_amount=0.03241129453953659
date=2018-01-05 00:00:00, price=15143.67, signal=0.0015734617254027738, usd_amount=452.7317174154743, btc_amount=0.042091188183812876
date=2018-01-06 00:00:00, price=16927.99, signal=0.0017843422890310048, usd_amount=573.9012400805854, btc_amount=0.03497432584516681
date=2018-01-07 00:00:00, price=17142.43, signal=0.0018876802999888902, usd_amount=614.8291206938326, btc_amount=0.033222380190007485
date=2018-01-08 00:00:00, price=16173.98, signal=-0.0012629230158462043, usd_amount=773.2385809883332, btc_amount=0.023995537301077882
date=2018-01-09 00:00:00, price=14999.99, signal=-0.0023063736

date=2018-03-07 00:00:00, price=10723.01, signal=0.0016531365581422975, usd_amount=336.8053970421833, btc_amount=0.061966853762004885
date=2018-03-08 00:00:00, price=9904.52, signal=0.002925605789906191, usd_amount=550.5577001065917, btc_amount=0.04248974570648007
date=2018-03-09 00:00:00, price=9293.06, signal=-0.0022418331310004773, usd_amount=547.3239955822307, btc_amount=0.04259482948900907
date=2018-03-10 00:00:00, price=9228.83, signal=-0.00217289051794927, usd_amount=543.6072177464338, btc_amount=0.04128491984103613
date=2018-03-11 00:00:00, price=8795.04, signal=0.0024785399666945518, usd_amount=414.17830482486784, btc_amount=0.05566572096950608
date=2018-03-12 00:00:00, price=9535.04, signal=0.002540494703274119, usd_amount=460.2506536946303, btc_amount=0.051199011395727856
date=2018-03-13 00:00:00, price=9115.24, signal=0.0027241617855482876, usd_amount=471.79638100792226, btc_amount=0.05000138649275899
date=2018-03-14 00:00:00, price=9152.07, signal=-0.0029189616124572, usd_

date=2018-05-09 00:00:00, price=9181.52, signal=-0.0021135842531731686, usd_amount=579.7340194468151, btc_amount=0.0401581008102902
date=2018-05-10 00:00:00, price=9308.49, signal=-0.0031059696239096692, usd_amount=407.3229108251901, btc_amount=0.059013422854283716
date=2018-05-11 00:00:00, price=9007.19, signal=0.0024767077518192617, usd_amount=423.8553686070698, btc_amount=0.05742489291421711
date=2018-05-12 00:00:00, price=8408.79, signal=0.002989086332819704, usd_amount=477.557386026469, btc_amount=0.05178447351385024
date=2018-05-13 00:00:00, price=8481.72, signal=0.0023572200454112513, usd_amount=379.87232766774486, btc_amount=0.06345941093504062
date=2018-05-14 00:00:00, price=8696.54, signal=0.0026822687705934305, usd_amount=443.2026954301153, btc_amount=0.0555654410486731
date=2018-05-15 00:00:00, price=8674.53, signal=0.0021212197473856, usd_amount=349.6111023704874, btc_amount=0.06585211993521807
date=2018-05-16 00:00:00, price=8481.99, signal=0.002619958193425466, usd_amoun

date=2018-07-12 00:00:00, price=6379.13, signal=-0.00414864458394165, usd_amount=283.87463616616253, btc_amount=0.07882424709489136
date=2018-07-13 00:00:00, price=6243.53, signal=0.0027781369387579885, usd_amount=329.5622451036296, btc_amount=0.07180841299999235
date=2018-07-14 00:00:00, price=6208.78, signal=-0.0035201043353826587, usd_amount=359.6872751916681, btc_amount=0.06688198237227051
date=2018-07-15 00:00:00, price=6245.99, signal=0, usd_amount=389.5151680466452, btc_amount=0.06210527558167042
date=2018-07-16 00:00:00, price=6353.25, signal=0.002375413934442212, usd_amount=286.74037300090464, btc_amount=0.07830760211702648
date=2018-07-17 00:00:00, price=6721.21, signal=-0.003779356788136705, usd_amount=325.8783645640841, btc_amount=0.07180777897459739
date=2018-07-18 00:00:00, price=7310.71, signal=-0.002884169372928705, usd_amount=449.70089100753876, btc_amount=0.05479921808564539
date=2018-07-19 00:00:00, price=7384.92, signal=0.002328616080844289, usd_amount=326.736225887

date=2018-09-21 00:00:00, price=6492.98, signal=0.0032914676576052044, usd_amount=406.0572397580713, btc_amount=0.05993658590648904
date=2018-09-22 00:00:00, price=6748.85, signal=-0.004024469557967718, usd_amount=296.2249575326036, btc_amount=0.07646492160138664
date=2018-09-23 00:00:00, price=6710.01, signal=-0.003156937429439226, usd_amount=407.64593001599474, btc_amount=0.05998181115934529
date=2018-09-25 00:00:00, price=6578.7, signal=0.0031161094742103795, usd_amount=389.4990385617686, btc_amount=0.06268176550293315
date=2018-09-26 00:00:00, price=6428.09, signal=0.0023734207897028897, usd_amount=289.87468643754374, btc_amount=0.07802415883600523
date=2018-09-27 00:00:00, price=6454.32, signal=0.0016155754500133952, usd_amount=198.1213778320787, btc_amount=0.0923347465158136
date=2018-09-28 00:00:00, price=6684.5, signal=-0.002872052805255188, usd_amount=447.75061819725744, btc_amount=0.054569003299848574
date=2018-09-29 00:00:00, price=6609.02, signal=-0.0031997403209019387, usd

...

date=2020-08-18 00:00:00, price=12301.27, signal=0.0030918516664769774, usd_amount=722.6403408363817, btc_amount=0.06102933869218403
date=2020-08-19 00:00:00, price=11950.0, signal=-0.0023687542357395627, usd_amount=911.1510234888954, btc_amount=0.04500633047905169
date=2020-08-20 00:00:00, price=11762.18, signal=0.003083835371266855, usd_amount=689.1799078169439, btc_amount=0.06352322238299547
date=2020-08-21 00:00:00, price=11857.18, signal=-0.002690470175078799, usd_amount=836.1493852502052, btc_amount=0.05111893332649718
date=2020-08-22 00:00:00, price=11527.81, signal=0.00354685293491725, usd_amount=776.8614879017001, btc_amount=0.055630049215097496
date=2020-08-23 00:00:00, price=11678.17, signal=-0.003837064104189386, usd_amount=574.3775023779563, btc_amount=0.07290421797959834
date=2020-08-24 00:00:00, price=11653.7, signal=0.0033855476603354253, usd_amount=749.628978615768, btc_amount=0.05789859893623896
date=2020-08-25 00:00:00, price=11750.4, signal=0.002616729304348953, usd

date=2020-10-29 00:00:00, price=13275.82, signal=-0.0016818459107598104, usd_amount=1092.5326183136563, btc_amount=0.0319550723044364
date=2020-10-30 00:00:00, price=13456.14, signal=0.0034000616328357484, usd_amount=869.2824014612621, btc_amount=0.0483127642593026
date=2020-11-01 00:00:00, price=13799.44, signal=0.0025592224369168524, usd_amount=671.00089283287, btc_amount=0.06300597378544891
date=2020-11-02 00:00:00, price=13762.0, signal=-0.0025167762535331538, usd_amount=881.4297535946507, btc_amount=0.04781874881712993
date=2020-11-03 00:00:00, price=13578.82, signal=-0.00247833420748241, usd_amount=889.4705802609552, btc_amount=0.04708834994216579
date=2020-11-04 00:00:00, price=14041.58, signal=0.0027101610413140945, usd_amount=723.043918415408, btc_amount=0.05942054015189661
date=2020-11-05 00:00:00, price=14168.49, signal=0.0023359524762741023, usd_amount=628.8414667107322, btc_amount=0.06573199656471584
date=2020-11-06 00:00:00, price=15608.32, signal=0.0027266814594979144, u

date=2021-01-02 00:00:00, price=29410.77, signal=0.0014876776448674992, usd_amount=831.3211558994543, btc_amount=0.04560096995139229
date=2021-01-03 00:00:00, price=32216.51, signal=0.0018087952058957898, usd_amount=1107.1883079351817, btc_amount=0.036735772110608626
date=2021-01-04 00:00:00, price=33068.83, signal=0.0018321661336726726, usd_amount=1151.164217717399, btc_amount=0.03575577235000724
date=2021-01-05 00:00:00, price=32054.23, signal=-0.0020909654892224344, usd_amount=1073.0692465025747, btc_amount=0.03972834429522626
date=2021-01-06 00:00:00, price=34081.24, signal=0.002159414664832154, usd_amount=1398.3150595816198, btc_amount=0.030347025135938375
date=2021-01-07 00:00:00, price=36875.96, signal=-0.0019142972716652547, usd_amount=1185.0813974472758, btc_amount=0.03637164816163984
date=2021-01-08 00:00:00, price=39510.59, signal=0.0013640190719722842, usd_amount=1023.9707677926708, btc_amount=0.03992797887712523
date=2021-01-09 00:00:00, price=40682.76, signal=0.0017794578

date=2021-03-06 00:00:00, price=48774.67, signal=-0.0011892099777185976, usd_amount=1821.706706285621, btc_amount=0.022594989576653356
date=2021-03-07 00:00:00, price=48933.12, signal=0.0016104174056784115, usd_amount=1497.2522150808575, btc_amount=0.028997876243093454
date=2021-03-08 00:00:00, price=50952.55, signal=-0.0016407736223271988, usd_amount=1388.7122222051385, btc_amount=0.031174698824216775
date=2021-03-09 00:00:00, price=52427.8, signal=0.0012480106834409327, usd_amount=1243.1786356767861, btc_amount=0.03382420658405742
date=2021-03-10 00:00:00, price=54927.29, signal=0.0007700818267908667, usd_amount=803.6716486535624, btc_amount=0.041581304669167075
date=2021-03-11 00:00:00, price=55918.09, signal=-0.0013750791887946758, usd_amount=1648.108353863934, btc_amount=0.02612650458709884
date=2021-03-12 00:00:00, price=57847.92, signal=-0.0012007500031157314, usd_amount=1842.851337170041, btc_amount=0.022814250059198897
date=2021-03-13 00:00:00, price=57297.67, signal=0.0014871

StopIteration: 